In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [22]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [23]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [24]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim),
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [25]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.01, random_state=42
)


In [26]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

In [27]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=20, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 15:53:02.414199: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 385704000 exceeds 10% of free system memory.


Epoch 1/20
1547/1547 [==============================] - 5s 3ms/step - loss: 0.0220 - val_loss: 0.0137
Epoch 2/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0131 - val_loss: 0.0120
Epoch 3/20
1547/1547 [==============================] - 4s 3ms/step - loss: 0.0116 - val_loss: 0.0110
Epoch 4/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0106 - val_loss: 0.0102
Epoch 5/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0100 - val_loss: 0.0098
Epoch 6/20
1547/1547 [==============================] - 4s 3ms/step - loss: 0.0095 - val_loss: 0.0094
Epoch 7/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0092 - val_loss: 0.0092
Epoch 8/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0090 - val_loss: 0.0089
Epoch 9/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.0087 - val_loss: 0.0088
Epoch 10/20
1547/1547 [==============================] - 4s 2ms/step - loss: 0.008

## PreModel

In [28]:
class ELumon(Model):
    def __init__(self, autoencoder):
        super(ELumon, self).__init__()
        self.encoder = autoencoder.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        return self.regressor(encoded)

In [29]:
elumon = ELumon(autoencoder)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
elumon.compile(optimizer=opt, loss="mean_squared_error")

In [30]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.15, random_state=42
)

In [31]:
elumon.fit(X_pretrain_train, y_pretrain_train, epochs=20, batch_size=32, validation_data=(X_pretrain_val, y_pretrain_val))

Epoch 1/20
1329/1329 [==============================] - 3s 2ms/step - loss: 9.4297 - val_loss: 2.8839
Epoch 2/20
1329/1329 [==============================] - 2s 2ms/step - loss: 1.8230 - val_loss: 0.1095
Epoch 3/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.1880 - val_loss: 0.1137
Epoch 4/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.1386 - val_loss: 0.0855
Epoch 5/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.1110 - val_loss: 0.0763
Epoch 6/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.0920 - val_loss: 0.0714
Epoch 7/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.0803 - val_loss: 0.0664
Epoch 8/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.0737 - val_loss: 0.0634
Epoch 9/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.0678 - val_loss: 0.0590
Epoch 10/20
1329/1329 [==============================] - 3s 2ms/step - loss: 0.064

## Model

In [37]:
class EGap(Model):
    def __init__(self, premodel):
        super(EGap, self).__init__()
        self.encoder = premodel.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        return self.regressor(encoded)

In [38]:
egap = EGap(elumon)
egap.compile(optimizer=opt, loss="mean_squared_error")

In [39]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    train_features[train_features.columns[2:]], train_labels[["homo_lumo_gap"]], test_size=0.02, random_state=42
)

In [40]:
egap.fit(X_train_train, y_train_train, epochs=20, batch_size=90, validation_data=(X_train_val, y_train_val))

Epoch 1/20
2/2 [==============================] - 1s 160ms/step - loss: 1.6336 - val_loss: 3.3093
Epoch 2/20
2/2 [==============================] - 0s 34ms/step - loss: 1.5729 - val_loss: 3.2944
Epoch 3/20
2/2 [==============================] - 0s 29ms/step - loss: 1.4649 - val_loss: 3.1854
Epoch 4/20
2/2 [==============================] - 0s 27ms/step - loss: 1.3731 - val_loss: 3.1214
Epoch 5/20
2/2 [==============================] - 0s 30ms/step - loss: 1.3163 - val_loss: 3.0642
Epoch 6/20
2/2 [==============================] - 0s 34ms/step - loss: 1.2247 - val_loss: 3.0033
Epoch 7/20
2/2 [==============================] - 0s 27ms/step - loss: 1.1620 - val_loss: 3.0210
Epoch 8/20
2/2 [==============================] - 0s 27ms/step - loss: 1.1203 - val_loss: 3.0473
Epoch 9/20
2/2 [==============================] - 0s 28ms/step - loss: 1.0870 - val_loss: 2.9725
Epoch 10/20
2/2 [==============================] - 0s 28ms/step - loss: 0.9807 - val_loss: 2.9076
Epoch 11/20
2/2 [===========

In [41]:
y_pred = egap.predict(test_features[test_features.columns[2:]])

In [42]:
output_df = pd.DataFrame({"Id": test_features["Id"], "y": y_pred.reshape(-1,)})
output_df.to_csv("submission.csv", index=False)